In [1]:
import json
import pandas as pd
# package for flattening json in pandas df
from pandas.io.json import json_normalize
from flatten_json import flatten

In [2]:
# load json object
with open('test.json') as scalar_json:
    scalar_data = json.load(scalar_json)

In [3]:
metric_catalogues = scalar_data.keys()
print(metric_catalogues)

dict_keys(['Ecosystem and Carbon Cycle', 'Hydrology Cycle', 'Radiation and Energy Cycle', 'Forcings', 'Relationships'])


In [4]:
modelNames = [
  "bcc-csm1-1",
  "bcc-csm1-1-m",
  "CESM1-BGC",
  "GFDL-ESM2G",
  "inmcm4",
  "IPSL-CM5A-LR",
  "MIROC-ESM",
  "MPI-ESM-LR",
  "NorESM1-ME",
  "MeanCMIP5",
  "BCC-CSM2-MR",
  "BCC-ESM1",
  "CESM2",
  "CESM2-WACCM",
  "CNRM-CM6-1",
  "CNRM-ESM2-1",
  "E3SMv1-CTC",
  "GISS-E2-1-G",
  "GISS-E2-1-H",
  "IPSL-CM6A-LR",
  "MIROC6",
  "MRI-ESM2-0",
  "MeanCMIP6"
]

In [5]:
metric_catalogues_list = list(metric_catalogues)
scalar_data[metric_catalogues_list[0]].keys()
regions = set([x.split()[-1] for x in scalar_data[metric_catalogues_list[0]].keys() if x != "children"])
print(regions)
scores = set([x.rsplit(' ', 1)[0] for x in scalar_data[metric_catalogues_list[0]].keys() if x != "children"])
print(scores)
children = scalar_data[metric_catalogues_list[0]]['children'].keys()
print(children)
# scalar_data[scalar_keys_list[0]]['children'].keys()
score_values = set([x.rsplit(' ', 1)[0] for x in scalar_data[metric_catalogues_list[0]].keys() if x != "children"])

{'globe', 'southamericaamazon', 'global'}
{'Overall Score', 'Spatial Distribution Score', 'Bias Score'}
dict_keys(['Biomass', 'Burned Area', 'Carbon Dioxide', 'Gross Primary Productivity', 'Leaf Area Index', 'Global Net Ecosystem Carbon Balance', 'Net Ecosystem Exchange', 'Ecosystem Respiration', 'Soil Carbon'])


In [6]:
obj = {}
for region in regions:
    obj[region] = {}
    for catalogue in metric_catalogues:
        print("catalogue:", catalogue)
        score_dict = {key.rsplit(' ', 1)[0]:value for (key,value) in scalar_data[catalogue].items() if key != 'children' and region in key}
#         print("score_dict:", score_dict.keys())
        for score, values in score_dict.items():
            score_dict[score] = {key:value for (key,value) in zip(modelNames, values)}
#         print("score_dict:", score_dict)
        metrics = scalar_data[catalogue]['children']
#         print("metrics:", metrics)
        metrics_obj = {}
        if metrics:
            for metric, value in metrics.items():
#                 print("metric:", metric)
#                 print("value:", value)
                metric_score_dict = {key.rsplit(' ', 1)[0]:value for (key,value) in value.items() if key != 'children' and region in key}
#                 print("metric_score_dict:", metric_score_dict)
                for score, values in metric_score_dict.items():
                    metric_score_dict[score] = {key:value for (key,value) in zip(modelNames, values)}
                metrics_obj[metric] = {"scores": metric_score_dict}
                observational_products = value['children']
                if observational_products:
                    products_obj = {}
                    for product, product_value in observational_products.items():
#                         print("product:", product)
                        product_score_dict = {key.rsplit(' ', 1)[0]:value for (key,value) in product_value.items() if key != 'children' and region in key}
                        for score, values in product_score_dict.items():
                            product_score_dict[score] = {key:value for (key,value) in zip(modelNames, values)}
#                         print("product_score_dict:", product_score_dict)
                        products_obj[product] = {"scores": product_score_dict}
#                 print("observational_products:", observational_products)
                metrics_obj[metric]["observational_products"] = products_obj
#             print("metric_obj:", metrics_obj)
#         obj[region].append({catalogue: {"scores": score_dict, "metrics": metrics_obj}})
        obj[region][catalogue] = {"scores": score_dict, "metrics": metrics_obj}

        

catalogue: Ecosystem and Carbon Cycle
catalogue: Hydrology Cycle
catalogue: Radiation and Energy Cycle
catalogue: Forcings
catalogue: Relationships
catalogue: Ecosystem and Carbon Cycle
catalogue: Hydrology Cycle
catalogue: Radiation and Energy Cycle
catalogue: Forcings
catalogue: Relationships
catalogue: Ecosystem and Carbon Cycle
catalogue: Hydrology Cycle
catalogue: Radiation and Energy Cycle
catalogue: Forcings
catalogue: Relationships


In [7]:
print(obj['global'].keys())

dict_keys(['Ecosystem and Carbon Cycle', 'Hydrology Cycle', 'Radiation and Energy Cycle', 'Forcings', 'Relationships'])


In [8]:
with open("data_file.json", "w") as write_file:
    json.dump(obj, write_file)

In [14]:
# load json object
with open('data_file.json') as data_json:
    data = json.load(data_json)

In [15]:
test_data = json_normalize(data, sep=".")
test_data.head()

,globe.Ecosystem and Carbon Cycle.scores.Spatial Distribution Score.bcc-csm1-1,globe.Ecosystem and Carbon Cycle.scores.Spatial Distribution Score.bcc-csm1-1-m,globe.Ecosystem and Carbon Cycle.scores.Spatial Distribution Score.CESM1-BGC,globe.Ecosystem and Carbon Cycle.scores.Spatial Distribution Score.GFDL-ESM2G,globe.Ecosystem and Carbon Cycle.scores.Spatial Distribution Score.inmcm4,globe.Ecosystem and Carbon Cycle.scores.Spatial Distribution Score.IPSL-CM5A-LR,globe.Ecosystem and Carbon Cycle.scores.Spatial Distribution Score.MIROC-ESM,globe.Ecosystem and Carbon Cycle.scores.Spatial Distribution Score.MPI-ESM-LR,globe.Ecosystem and Carbon Cycle.scores.Spatial Distribution Score.NorESM1-ME,globe.Ecosystem and Carbon Cycle.scores.Spatial Distribution Score.MeanCMIP5,...,global.Relationships.metrics.Evapotranspiration/MODIS.observational_products.SurfaceAirTemperature/CRU.scores.Overall Score.CESM2-WACCM,global.Relationships.metrics.Evapotranspiration/MODIS.observational_products.SurfaceAirTemperature/CRU.scores.Overall Score.CNRM-CM6-1,global.Relationships.metrics.Evapotranspiration/MODIS.observational_products.SurfaceAirTemperature/CRU.scores.Overall Score.CNRM-ESM2-1,global.Relationships.metrics.Evapotranspiration/MODIS.observational_products.SurfaceAirTemperature/CRU.scores.Overall Score.E3SMv1-CTC,global.Relationships.metrics.Evapotranspiration/MODIS.observational_products.SurfaceAirTemperature/CRU.scores.Overall Score.GISS-E2-1-G,global.Relationships.metrics.Evapotranspiration/MODIS.observational_products.SurfaceAirTemperature/CRU.scores.Overall Score.GISS-E2-1-H,global.Relationships.metrics.Evapotranspiration/MODIS.observational_products.SurfaceAirTemperature/CRU.scores.Overall Score.IPSL-CM6A-LR,global.Relationships.metrics.Evapotranspiration/MODIS.observational_products.SurfaceAirTemperature/CRU.scores.Overall Score.MIROC6,global.Relationships.metrics.Evapotranspiration/MODIS.observational_products.SurfaceAirTemperature/CRU.scores.Overall Score.MRI-ESM2-0,global.Relationships.metrics.Evapotranspiration/MODIS.observational_products.SurfaceAirTemperature/CRU.scores.Overall Score.MeanCMIP6
0,-999,-999,-1.699756,-999,-999,-999,-999,0.008352,-1.834848,1.000802,...,0.809805,0.340529,0.556433,0.195735,-0.105734,-0.424612,0.602475,-0.81772,-0.325667,1.034749


In [17]:
test_data.to_json("normalized_data.json")

In [18]:
flatten_json = flatten(data)

In [19]:
with open("flattened_data.json", "w") as write_file:
    json.dump(flatten_json, write_file)

In [10]:
children_data = json_normalize(data=scalar_data['Ecosystem and Carbon Cycle'], record_path=['children'])
children_data.head(3)

,0
0,Biomass
1,Burned Area
2,Carbon Dioxide


In [14]:
global_columns = [col for col in test_data.columns if 'global' in col]
print(global_columns)

['Spatial Distribution Score global', 'Bias Score global', 'Overall Score global', 'children.Biomass.children.GEOCARBON.Spatial Distribution Score global', 'children.Biomass.children.GEOCARBON.Bias Score global', 'children.Biomass.children.GEOCARBON.Overall Score global', 'children.Biomass.children.Tropical.Spatial Distribution Score global', 'children.Biomass.children.Tropical.Bias Score global', 'children.Biomass.children.Tropical.Overall Score global', 'children.Biomass.children.GlobalCarbon.Spatial Distribution Score global', 'children.Biomass.children.GlobalCarbon.Bias Score global', 'children.Biomass.children.GlobalCarbon.Overall Score global', 'children.Biomass.children.NBCD2000.Spatial Distribution Score global', 'children.Biomass.children.NBCD2000.Bias Score global', 'children.Biomass.children.NBCD2000.Overall Score global', 'children.Biomass.children.USForest.Spatial Distribution Score global', 'children.Biomass.children.USForest.Bias Score global', 'children.Biomass.children

In [35]:
class Region(object):
    def __init__(self, name):
        self.name = name
        self.metric_catalogue = None
    
    def __str__(self):
        return str(self.name)
    

In [36]:
class MetricCatalogue(object):
    def __init__(self, name):
        self.name = name
        self.scores = None
        self.metrics = None
    
    def __str__(self):
        return str(self.name)

In [47]:
class Metric(object):
    def __init__(self, name, scores, region):
        self.name = name
        self.scores = scores
        self.region = region
#         self.observational_products = None
    def __str__(self):
        return str(self.name)

In [53]:
child = scalar_data[metric_catalogues_list[0]]['children']["Biomass"]['children']['GEOCARBON']
# print(child)
score_dict = {key.rsplit(' ', 1)[0]:value for (key,value) in child.items() if key != 'children' and 'global' in key}
print(score_dict)

{'Spatial Distribution Score': [-1.3781402005154966, -1.6027807377679966, 1.119973955423911, -0.006327760617965289, -0.2819375165374983, 0.8525210991935368, -1.9146281936454548, -1.1555870084686382, 0.8967565756814948, 0.016264537911517083, -0.5925032946972298, -0.2870022816376711, 1.3027858950196538, 1.3234510402767277, -999, 0.22828244873706444, 1.1137080115760354, -999, -999, -0.27796506596147263, -999, -999, 0.6431284960294728], 'Bias Score': [-0.6399375925766302, -1.0108122356871663, 0.45124530812540053, -0.7498943745538078, -0.6733906730322314, -0.5467566600419714, -1.8194448417286586, -1.139378392313906, 0.07999679224675713, 0.7383679685301003, -0.2912656679302592, 0.0366710310513623, 1.72366445294885, 1.7751149240951982, -999, -0.1979619181067939, 0.5132187468894708, -999, -999, -0.08575348101982856, -999, -999, 1.8363166131041355], 'Overall Score': [-1.2648740926394078, -1.5313595294557112, 1.0123575551343686, -0.1747999980729382, -0.381495600631861, 0.5691939959917459, -1.967

In [54]:
geocarbon = Metric('GEOCARBON', score_dict, 'global')

In [55]:
geocarbon.__dict__

{'name': 'GEOCARBON',
 'scores': {'Spatial Distribution Score': [-1.3781402005154966,
   -1.6027807377679966,
   1.119973955423911,
   -0.006327760617965289,
   -0.2819375165374983,
   0.8525210991935368,
   -1.9146281936454548,
   -1.1555870084686382,
   0.8967565756814948,
   0.016264537911517083,
   -0.5925032946972298,
   -0.2870022816376711,
   1.3027858950196538,
   1.3234510402767277,
   -999,
   0.22828244873706444,
   1.1137080115760354,
   -999,
   -999,
   -0.27796506596147263,
   -999,
   -999,
   0.6431284960294728],
  'Bias Score': [-0.6399375925766302,
   -1.0108122356871663,
   0.45124530812540053,
   -0.7498943745538078,
   -0.6733906730322314,
   -0.5467566600419714,
   -1.8194448417286586,
   -1.139378392313906,
   0.07999679224675713,
   0.7383679685301003,
   -0.2912656679302592,
   0.0366710310513623,
   1.72366445294885,
   1.7751149240951982,
   -999,
   -0.1979619181067939,
   0.5132187468894708,
   -999,
   -999,
   -0.08575348101982856,
   -999,
   -999,
   1

In [14]:
def extract_values(obj, key):
    """Recursively pull values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Return all matching values in an object."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [17]:
names = extract_values(scalar_data, 'Bias Score global')

In [18]:
names

[]